# 목차
## 1. Library Import
## 2. Data Set
## 3. Shape Set
## 4. AutoML Set
## 5. Model Train
## 6. Prediction
## 7. Submission

## 1. Library Import

In [1]:
!pip install pycaret

     |████████████████████████████████| 256kB 8.4MB/s 
     |████████████████████████████████| 276kB 14.5MB/s 
     |████████████████████████████████| 65.8MB 55kB/s 
     |████████████████████████████████| 157.5MB 78kB/s 
     |████████████████████████████████| 6.8MB 21.9MB/s 
     |████████████████████████████████| 102kB 18.4MB/s 
     |████████████████████████████████| 1.8MB 55.9MB/s 
     |████████████████████████████████| 14.2MB 16.0MB/s 
     |████████████████████████████████| 174kB 79.1MB/s 
     |████████████████████████████████| 245kB 45.9MB/s 
     |████████████████████████████████| 1.6MB 51.0MB/s 
     |████████████████████████████████| 2.1MB 56.7MB/s 
     |████████████████████████████████| 348kB 56.5MB/s 
     |████████████████████████████████| 61kB 11.6MB/s 
     |████████████████████████████████| 1.1MB 48.0MB/s 
     |████████████████████████████████| 81kB 14.1MB/s 
     |████████████████████████████████| 153kB 82.2MB/s 
     |████████████████████████████████| 163kB 78.0M

In [2]:
import pandas as pd
import numpy as np

import datetime as dt

import matplotlib.pyplot as plt

from tqdm import tqdm

import gc
import random
import lightgbm as lgb
import re

from sklearn.metrics import *
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings(action='ignore')

# 모듈
import os, pathlib, glob
import random
from pytz import timezone # Colab 사용 시, Asia/Seoul로 지정 필요

from pycaret.classification import *

In [3]:
# HDLY
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
# HDLY
# 작업경로 변경
os.chdir('/content/gdrive/MyDrive/Colab/Dacon_LG_AUC')

print('Current directory: ', os.getcwd())

Current directory:  /content/gdrive/MyDrive/Colab/Dacon_LG_AUC


In [ ]:
########################################
########################################
########################################

## 2. Data Set

#### 2.1. train_err

In [6]:
PATH = os.getcwd() + '/'

# load
train_err_1 = pd.read_csv(PATH+'HDLY_train_err_with_DayHour.csv')
train_err_2 = pd.read_csv(PATH+'HDLY_train_err_with_ErrFwverModelQuality.csv')

# concat
train_err = pd.concat([train_err_1, train_err_2], axis=1)

# 확인
train_err

,day_0,day_1,day_2,day_3,day_4,day_5,day_6,hour_0,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,fw_ctgr_3,fw_ctgr_4,fw_ctgr_5,fw_ctgr_8,fw_ctgr_9,err0,err1,err2,err3,err4,err5,err6,err7,err8,err9,err10,err11,err12,err13,err14,err15,err16,err17,err18,err19,err20,err21,err22,err23,err24,err25,err26,err27,err28,err29,err30,err31,err32,err33,err34,err35,err36,err37,err38,err39,err40,err41,model_0,model_1,model_2,model_3,model_4,model_5,model_6,model_7,model_8,user_id,qual_0,qual_1,qual_2,qual_3,qual_4,qual_5,qual_6,qual_7,qual_8,qual_9,qual_10,qual_11,qual_12,problem
0,54.0,52.0,37.0,41.0,45.0,45.0,43.0,70.0,89.0,27.0,10.0,40.0,31.0,50.0,0.0,0.0,317.0,0.0,0.0,0.0,0.0,8.0,104.0,0.0,1.0,1.0,0.0,0.0,7.0,15.0,16.0,1.0,10.0,59.0,61.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,10000,0.001172,0.108329,0.000439,0.0,0.0,9.729505e-07,0.004748,0.000000,0.0,0.000000e+00,9.729511e-07,0.956884,0.0,0.0
1,117.0,139.0,164.0,138.0,107.0,166.0,1534.0,150.0,24.0,114.0,412.0,739.0,491.0,435.0,0.0,2365.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,53.0,1.0,1.0,0.0,0.0,0.0,10.0,11.0,1.0,15.0,151.0,128.0,0.0,4.0,1.0,3.0,1.0,756.0,751.0,5.0,1.0,22.0,0.0,0.0,0.0,0.0,250.0,0.0,10.0,18.0,0.0,1.0,1.0,0.0,0.0,113.0,56.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,10001,0.001172,0.108329,0.000439,0.0,0.0,0.000000e+00,0.004748,0.000000,0.0,0.000000e+00,0.000000e+00,0.956884,0.0,1.0
2,42.0,58.0,52.0,34.0,46.0,40.0,34.0,46.0,45.0,45.0,14.0,42.0,57.0,57.0,0.0,0.0,306.0,0.0,0.0,0.0,0.0,2.0,132.0,1.0,2.0,1.0,0.0,0.0,1.0,13.0,14.0,1.0,4.0,52.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,25.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,10002,0.001172,0.108159,0.000438,0.0,0.0,2.675614e-06,0.004938,0.000200,0.0,4.764160e-07,2.513457e-06,0.955371,0.0,0.0
3,22.0,57.0,51.0,52.0,42.0,45.0,37.0,78.0,62.0,16.0,4.0,16.0,63.0,67.0,0.0,306.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,9.0,9.0,0.0,0.0,52.0,30.0,1.0,0.0,0.0,0.0,0.0,28.0,19.0,0.0,0.0,59.0,0.0,0.0,0.0,0.0,65.0,0.0,8.0,0.0,0.0,1.0,1.0,2.0,0.0,17.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,10003,0.001172,0.108329,0.000439,0.0,0.0,0.000000e+00,0.004748,0.000000,0.0,0.000000e+00,0.000000e+00,0.956884,0.0,0.0
4,84.0,108.0,96.0,144.0,115.0,59.0,171.0,138.0,98.0,70.0,55.0,55.0,223.0,138.0,0.0,777.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0,4.0,0.0,0.0,0.0,16.0,19.0,3.0,5.0,143.0,91.0,0.0,0.0,0.0,0.0,0.0,140.0,119.0,0.0,0.0,33.0,0.0,0.0,0.0,0.0,176.0,0.0,16.0,0.0,0.0,1.0,1.0,0.0,0.0,4.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10004,0.001169,0.108159,0.000438,0.0,0.0,4.864753e-07,0.005115,0.000378,0.0,0.000000e+00,3.243170e-07,0.955371,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,39.0,34.0,40.0,0.0,24.0,3.0,54.0,38.0,8.0,7.0,18.0,31.0,35.0,57.0,0.0,194.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,5.0,5.0,0.0,0.0,0.0,5.0,10.0,8.0,8.0,26.0,18.0,0.0,0.0,0.0,0.0,0.0,17.0,12.0,0.0,0.0,22.0,0.0,0.0,0.0,0.0,31.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,7.0,4.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,24995,0.001159,0.107477,0.000434,0.0,0.0,1.216188e-06,0.004705,0.000000,0.0,0.000000e+00,4.053963e-07,0.949319,0.0,0.0
14996,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,24996,0.001172,0.108329,0.000439,0.0,0.0,0.000000e+00,0.004748,0.000000,0.0,0.000000e+00,0.000000e+00,0.956884,0.0,0.0
14997,118.0,133.0,125.0,112.0,133.0,96.0,109.0,170.0,144.0,132.0,55.0,43.0,119.0,163.0,0.0,826.0,0.0

In [ ]:
########################################
########################################
########################################

#### 2.2. test_err



In [7]:
# load
test_err_1 = pd.read_csv(PATH+'HDLY_test_err_with_DayHour.csv')
test_err_2 = pd.read_csv(PATH+'HDLY_test_err_with_ErrFwverModelQuality.csv')

# concat
test_err = pd.concat([test_err_1, test_err_2], axis=1)

# 확인
test_err

,day_0,day_1,day_2,day_3,day_4,day_5,day_6,hour_0,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,fw_ctgr_3,fw_ctgr_4,fw_ctgr_5,fw_ctgr_8,fw_ctgr_9,err0,err1,err2,err3,err4,err5,err6,err7,err8,err9,err10,err11,err12,err13,err14,err15,err16,err17,err18,err19,err20,err21,err22,err23,err24,err25,err26,err27,err28,err29,err30,err31,err32,err33,err34,err35,err36,err37,err38,err39,err40,err41,model_0,model_1,model_2,model_3,model_4,model_5,model_6,model_7,model_8,qual_0,qual_1,qual_2,qual_3,qual_4,qual_5,qual_6,qual_7,qual_8,qual_9,qual_10,qual_11,qual_12
0,1024.0,510.0,267.0,62.0,410.0,254.0,223.0,267.0,123.0,165.0,74.0,679.0,1140.0,302.0,0.0,2750.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,62.0,1.0,1.0,0.0,0.0,0.0,16.0,16.0,0.0,3.0,67.0,37.0,9.0,3.0,0.0,1.0,0.0,983.0,1308.0,5.0,2.0,32.0,0.0,0.0,0.0,0.0,83.0,2.0,16.0,0.0,1.0,3.0,3.0,0.0,2.0,92.0,0.0,2.0,0.0,2687.0,63.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001172,0.108329,0.000439,0.0,0.0,1.621584e-07,0.004769,0.000022,0.0,0.0,1.621585e-07,0.956884,0.0
1,54.0,56.0,43.0,32.0,57.0,24.0,18.0,36.0,44.0,43.0,38.0,32.0,45.0,46.0,0.0,0.0,284.0,0.0,0.0,0.0,0.0,3.0,98.0,3.0,4.0,4.0,0.0,0.0,0.0,12.0,16.0,4.0,6.0,53.0,55.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,284.0,0.0,0.0,0.0,0.0,0.0,0.001160,0.109096,0.000434,0.0,0.0,2.085357e-04,0.004709,0.000000,0.0,0.0,2.078061e-04,0.950076,0.0
2,140.0,135.0,138.0,120.0,158.0,151.0,99.0,178.0,131.0,94.0,115.0,200.0,140.0,83.0,0.0,941.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,79.0,2.0,1.0,0.0,0.0,0.0,16.0,16.0,0.0,1.0,173.0,133.0,0.0,0.0,0.0,0.0,0.0,60.0,40.0,2.0,0.0,40.0,0.0,0.0,0.0,0.0,230.0,0.0,16.0,0.0,0.0,1.0,1.0,0.0,14.0,113.0,1.0,2.0,941.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001153,0.109607,0.000431,0.0,0.0,1.199972e-05,0.004683,0.000000,0.0,0.0,1.078354e-05,0.945537,0.0
3,24.0,71.0,49.0,34.0,69.0,32.0,92.0,91.0,34.0,33.0,41.0,59.0,54.0,59.0,0.0,371.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,107.0,2.0,2.0,0.0,0.0,0.0,4.0,5.0,1.0,7.0,8.0,12.0,0.0,1.0,0.0,1.0,0.0,3.0,2.0,0.0,0.0,5.0,2.0,2.0,0.0,0.0,134.0,0.0,4.0,16.0,3.0,1.0,1.0,0.0,0.0,40.0,8.0,0.0,371.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001172,0.108329,0.000439,0.0,0.0,9.729505e-07,0.005698,0.000955,0.0,0.0,9.729511e-07,0.956884,0.0
4,102.0,157.0,109.0,112.0,120.0,176.0,105.0,129.0,119.0,153.0,53.0,187.0,90.0,150.0,0.0,881.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,6.0,1.0,1.0,0.0,0.0,0.0,10.0,12.0,1.0,18.0,80.0,67.0,6.0,0.0,0.0,0.0,0.0,10.0,8.0,0.0,0.0,14.0,1.0,1.0,0.0,0.0,332.0,152.0,11.0,6.0,1.0,1.0,1.0,1.0,0.0,48.0,90.0,0.0,0.0,623.0,258.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001164,0.108840,0.000436,0.0,0.0,6.486337e-07,0.004864,0.000143,0.0,0.0,1.621585e-07,0.952345,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14994,116.0,245.0,177.0,133.0,151.0,139.0,154.0,105.0,26.0,61.0,182.0,299.0,205.0,237.0,0.0,1115.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,12.0,5.0,5.0,0.0,0.0,0.0,16.0,19.0,3.0,9.0,123.0,93.0,12.0,0.0,0.0,0.0,0.0,197.0,204.0,0.0,0.0,31.0,0.0,0.0,0.0,0.0,179.0,2.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,87.0,0.0,0.0,1115.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001158,0.109266,0.000433,0.0,0.0,1.054030e-06,0.004700,0.000000,0.0,0.0,1.621585e-07,0.948563,0.0
14995,71.0,78.0,83.0,86.0,55.0,68.0,74.0,105.0,9.0,42.0,105.0,60.0,115.0,79.0,0.0,515.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,14.0,15.0,1.0,4.0,149.0,98.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,183.0,0.0,14.0,1.0,0.0,1.0,1.0,0.0,0.0,12.0,17.0,0.0,515.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001172,0.108329,0.000439,0.0,0.0,0.000000e+00,0.004748,0.000000,0.0,0.0,0.000000e+00,0.956884,0.0
14996,419.0,533.0,223.0,204.0,169.0,272.0,413.0,78.0,306.0,223.0,193.0,428.0,505.0,500.0,0.0,2233.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,12.0,13.0,1

In [ ]:
########################################
########################################
########################################

#### 2.3. problem

In [8]:
train_prob = pd.read_csv(PATH+'train_problem_data.csv')

problem = np.zeros(15000)
problem[train_prob.user_id.unique()-10000] = 1 

problem.shape

(15000,)

In [ ]:
########################################
########################################
########################################

## 3. Shape sync.

In [9]:
train_err['problem'] = problem
train_err.drop(['user_id'], axis=1, inplace=True)

In [ ]:
########################################
########################################
########################################

## 4. AutoML

In [10]:
# AutoML

clf2 = setup(data = train_err, target = 'problem') 

,Description,Value
0,session_id,2046
1,Target,problem
2,Target Type,Binary
3,Label Encoded,"0.0: 0, 1.0: 1"
4,Original Data,"(15000, 84)"
5,Missing Values,False
6,Numeric Features,74
7,Categorical Features,9
8,Ordinal Features,False
9,High Cardinality Features,False


In [28]:
best_3 = compare_models(sort = 'Accuracy', n_select = 3)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
catboost,CatBoost Classifier,0.7924,0.8173,0.5156,0.8001,0.6266,0.4921,0.5151,14.535
et,Extra Trees Classifier,0.7882,0.8093,0.5075,0.7923,0.6182,0.4813,0.5044,1.512
lightgbm,Light Gradient Boosting Machine,0.7876,0.8132,0.5215,0.7789,0.6239,0.4840,0.5034,0.484
gbc,Gradient Boosting Classifier,0.7849,0.8115,0.4847,0.8027,0.6037,0.4683,0.4968,3.913
xgboost,Extreme Gradient Boosting,0.7808,0.7933,0.5373,0.7442,0.6237,0.4748,0.4875,5.383
rf,Random Forest Classifier,0.7800,0.8044,0.4726,0.7949,0.5922,0.4547,0.4838,2.241
ada,Ada Boost Classifier,0.7740,0.7928,0.4804,0.7651,0.5897,0.4449,0.4681,0.875
ridge,Ridge Classifier,0.7659,0.0000,0.3769,0.8461,0.5212,0.3952,0.4519,0.060
lda,Linear Discriminant Analysis,0.7655,0.7669,0.3819,0.8362,0.5240,0.3961,0.4497,0.152
lr,Logistic Regression,0.7309,0.7449,0.3284,0.7273,0.4517,0.3060,0.3487,1.638


In [29]:
blended = blend_models(estimator_list = best_3, fold = 5, method = 'soft')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7962,0.8217,0.5056,0.8234,0.6265,0.4972,0.5251
1,0.7900,0.8219,0.4754,0.8325,0.6052,0.4763,0.5110
2,0.8000,0.8273,0.5513,0.7951,0.6512,0.5173,0.5344
3,0.8000,0.8225,0.5316,0.8129,0.6429,0.5123,0.5345
4,0.7856,0.8069,0.4901,0.7982,0.6073,0.4712,0.4977
Mean,0.7944,0.8201,0.5108,0.8124,0.6266,0.4949,0.5206
SD,0.0057,0.0069,0.0275,0.0143,0.0184,0.0185,0.0143


In [30]:
pred_holdout = predict_model(blended)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.7952,0.8159,0.4921,0.792,0.607,0.4785,0.5033


In [31]:
final_model = finalize_model(blended)

In [ ]:
########################################
########################################
########################################

## 5. Model Train5

In [33]:
# 변수 이름 변경 : error  -> train_x / problem-> train_y
# train_err.drop(['problem'], axis=1, inplace=True)
train_x = train_err.to_numpy()
train_y = problem

print(train_x.shape)
print(train_y.shape)

(15000, 83)
(15000,)


In [34]:
# Train

#-------------------------------------------------------------------------------------

# validation auc score를 확인하기 위해 정의
def f_pr_auc(probas_pred, y_true):
    labels=y_true.get_label()
    p, r, _ = precision_recall_curve(labels, probas_pred)
    score=auc(r,p) 
    return "pr_auc", score, True

#-------------------------------------------------------------------------------------

models     = []
recalls    = []
precisions = []
auc_scores   = []
threshold = 0.5

lr = 0.01 #learning_rate
nbr = 20000 #num_boost_round 
vb = 500 #verbose_eval
esr = 1000 #early_stopping_rounds
boosting_type = ['gbdt','dart']

# 파라미터 설정
params =      {
                'boosting_type' : boosting_type[1],
                'objective'     : 'binary',
                'metric'        : 'auc',
                'learning_rate' : lr,
                'seed': 2021
                }

es_only_dart = [8000, 12000, 5500, 10500]
es_only_index = 0

#-------------------------------------------------------------------------------------

# 5 Kfold cross validation
k_fold = KFold(n_splits=4, shuffle=True, random_state=42)
for train_idx, val_idx in k_fold.split(train_x):

    # split train, validation set
    X = train_x[train_idx]
    y = train_y[train_idx]
    valid_x = train_x[val_idx]
    valid_y = train_y[val_idx]

    d_train= lgb.Dataset(X, y)
    d_val  = lgb.Dataset(valid_x, valid_y)
    
    #run traning
    model = lgb.train(
                        params,
                        train_set       = d_train,
                        # num_boost_round = nbr,
                        num_boost_round = es_only_dart[es_only_index],
                        valid_sets      = d_val,
                        feval           = f_pr_auc,
                        verbose_eval    = vb, 
                        early_stopping_rounds = esr
                       )
    
    # cal valid prediction
    valid_prob = model.predict(valid_x)
    valid_pred = np.where(valid_prob > threshold, 1, 0)
    
    # cal scores
    recall    = recall_score(    valid_y, valid_pred)
    precision = precision_score( valid_y, valid_pred)
    auc_score = roc_auc_score(   valid_y, valid_prob)

    # append scores
    models.append(model)
    recalls.append(recall)
    precisions.append(precision)
    auc_scores.append(auc_score)

    print('==========================================================')
    es_only_index += 1


print(f"RESULT AUC : {np.mean(auc_scores)}")

[LightGBM] [Info] Number of positive: 3610, number of negative: 7640
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005421 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 10366
[LightGBM] [Info] Number of data points in the train set: 11250, number of used features: 79
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.320889 -> initscore=-0.749690
[LightGBM] [Info] Start training from score -0.749690
[500]	valid_0's auc: 0.79671	valid_0's pr_auc: 0.741605
[1000]	valid_0's auc: 0.806185	valid_0's pr_auc: 0.747054
[1500]	valid_0's auc: 0.811161	valid_0's pr_auc: 0.748561
[2000]	valid_0's auc: 0.812927	valid_0's pr_auc: 0.751297
[2500]	valid_0's auc: 0.813487	valid_0's pr_auc: 0.753589
[3000]	valid_0's auc: 0.811455	valid_0's pr_auc: 0.752514
[3500]	valid_0's auc: 0.809058	valid_0's pr_auc: 0.75138
[4000]	valid_0's auc: 0.808602	valid_

In [ ]:
########################################
########################################
########################################

## 6. Prediction

In [35]:
# 예측

t_e_corr = train_err.corr()
t_list = list(t_e_corr.index)

pred_y_list = []

for model in models:
    pred_y = model.predict(test_err[t_list].to_numpy())
    pred_y_list.append(pred_y.reshape(-1,1))
    
pred_ensemble = np.mean(pred_y_list, axis = 0)

pred_ensemble

array([[0.95389095],
       [0.3367271 ],
       [0.24941807],
       ...,
       [0.68132364],
       [0.94013986],
       [0.38684661]])

In [36]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(250,250))
sns.heatmap(data = train_err.corr(), annot=True, fmt = '.2f', linewidths=.5, cmap='Blues')

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
########################################
########################################
########################################

## 7. submission to_csv

In [ ]:
# HDLY

# 제출

sample_submssion = pd.read_csv(PATH+'sample_submission.csv')
sample_submssion['problem'] = pred_ensemble.reshape(-1)

sample_submssion.to_csv("HDLY_0203_2_dart_.csv", index = False)
sample_submssion

In [ ]:
########################################
########################################
########################################